<a href="https://colab.research.google.com/github/nssn96/ML_imageclassifier/blob/main/ML_imageClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML Assignment 5 - Image Classification


1.   Author : Surya Narayanan Nadhamuni Suresh
2.   UTA ID : 1001877873


In [26]:
#import lines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import torch
import torchvision
import torchvision.transforms as transforms
import tensorflow_datasets as tfds
from torchvision.datasets import Food101

from torch import nn
import torchmetrics
from torch.utils.data import DataLoader, random_split


In [4]:
#References
#https://www.geeksforgeeks.org/training-neural-networks-using-pytorch-lightning/
#https://www.kaggle.com/code/uzairshahmdn/food101-midterm-exam
#https://www.kaggle.com/code/darthgera/food101-resnet-50-pytorch
#https://www.kaggle.com/code/ajaykrishnabandari/food-101
#https://www.kaggle.com/code/odednir/multiclass-food-classification-using-tensorflow

In [ ]:
#Installing pytorch lighting
#pip install pytorch-lightning lightning-bolts
!pip install pytorch-lightning


In [10]:
import pytorch_lightning as pl

In [ ]:
#Reference Used for loading the dataset
#https://colab.research.google.com/drive/1Z5-LqlJxNslHcCzM9jChsNhGi9Y7HSC4#scrollTo=f88XvEBTQBS9
#Loading the food101 dataset
#food, info = tfds.load("food101", with_info=True)
#ds = tfds.load('huggingface:food101')
import os
def get_data_extract():
  if "food-101" in os.listdir():
    print("Dataset already exists")
  else:
    print("Downloading the data...")
    !wget http://data.vision.ee.ethz.ch/cvl/food-101.tar.gz
    print("Dataset downloaded!")
    print("Extracting data..")
    !tar xzvf food-101.tar.gz
    print("Extraction done!")

# get_data_extract()

In [18]:
#contents inside the folder
!ls food-101/

images	license_agreement.txt  meta  README.txt


In [21]:
!ls food-101/meta

classes.txt  labels.txt  test.json  test.txt  train.json  train.txt


From above output we can see that we have the train dataset in train.txt and the test dataset in the test.txt

In [24]:
#checking some of the data in the train.txt file
!head 'food-101/meta/train.txt'

apple_pie/1005649
apple_pie/1014775
apple_pie/1026328
apple_pie/1028787
apple_pie/1043283
apple_pie/1050519
apple_pie/1057749
apple_pie/1057810
apple_pie/1072416
apple_pie/1074856


image ids are unique and correspond to the foodspotting.com review ids

In [ ]:
#The image folder contains 101 folders of food class with 1000 images in each
os.listdir('food-101/images')

Visualizing some of the images from the dataset

In [ ]:
#References
#https://www.kaggle.com/code/odednir/multiclass-food-classification-using-tensorflow
# Visualize the data, showing one image per class from 101 classes
rows = 17
cols = 6
fig, ax = plt.subplots(rows, cols, figsize=(25,25))
fig.suptitle("Showing one random image from each class", y=1.05, fontsize=24) # Adding  y=1.05, fontsize=24 helped me fix the suptitle overlapping with axes issue
data_dir = "food-101/images/"
foods_sorted = sorted(os.listdir(data_dir))
food_id = 0
for i in range(rows):
  for j in range(cols):
    try:
      food_selected = foods_sorted[food_id] 
      food_id += 1
    except:
      break
    if food_selected == '.DS_Store':
        continue
    food_selected_images = os.listdir(os.path.join(data_dir,food_selected)) # returns the list of all files present in each food category
    food_selected_random = np.random.choice(food_selected_images) # picks one food item from the list as choice, takes a list and returns one random item
    img = plt.imread(os.path.join(data_dir,food_selected, food_selected_random))
    ax[i][j].imshow(img)
    ax[i][j].set_title(food_selected, pad = 10)
    
plt.setp(ax, xticks=[],yticks=[])
plt.tight_layout()
# https://matplotlib.org/users/tight_layout_guide.html

A Basic CNN using Pytorch Lightining

In [28]:
#Reference
#https://www.kaggle.com/code/shivanandmn/cnn-pytorch-lightning-beginners-model
class_num=5
img_size = 64
class Food101Model(pl.LightningModule):
    def __init__(self):
      #image_size = 64
      super().__init__()
      self.cnv = nn.Conv2d(3,128,5,4)
      self.rel = nn.ReLU()
      self.bn = nn.BatchNorm2d(128)
      self.mxpool = nn.MaxPool2d(4)
      self.flat = nn.Flatten()
      self.fc1 = nn.Linear(1152,64)
      self.fc2 = nn.Linear(64,64)
      self.fc3 = nn.Linear(64,class_num)
      self.softmax = nn.Softmax()
      self.accuracy = pl.metrics.Accuracy()

    def forward(self,x):
      out = self.bn(self.rel(self.cnv(x)))
      out = self.flat(self.mxpool(out))
      out = self.rel(self.fc1(out))
      out = self.rel(self.fc2(out))
      out = self.fc3(out)
      return out

    def loss_fn(self,out,target):
      return nn.CrossEntropyLoss()(out.view(-1,class_num),target)
    
    def configure_optimizers(self):
      LR = 1e-3
      optimizer = torch.optim.AdamW(self.parameters(),lr=LR)
      return optimizer

    def training_step(self,batch,batch_idx):
      x,y = batch["x"],batch["y"]
      img = x.view(-1,3,img_size,img_size)
      label = y.view(-1)
      out = self(img)
      loss = self.loss_fn(out,label)
      self.log('train_loss', loss)
      return loss       

    def validation_step(self,batch,batch_idx):
      x,y = batch["x"],batch["y"]
      img = x.view(-1,3,img_size,img_size)
      label = y.view(-1)
      out = self(img)
      loss = self.loss_fn(out,label)
      out = nn.Softmax(-1)(out) 
      logits = torch.argmax(out,dim=1)
      accu = self.accuracy(logits, label)        
      self.log('valid_loss', loss)
      self.log('train_acc_step', accu)
      return loss, accu

In [31]:
#Getting the dataset
x_train = pd.read_csv('food-101/meta/train.txt', header = None, names=['path'])
#x_train.head()
x_test = pd.read_csv('food-101/meta/test.txt', header = None, names=['path'])
x_test.head()

,path
0,apple_pie/1011328
1,apple_pie/101251
2,apple_pie/1034399
3,apple_pie/103801
4,apple_pie/1038694


In [33]:
#references
#https://www.kaggle.com/code/uzairshahmdn/food101-midterm-exam
def split(data, id='id'):
    if id.upper() == 'CLASS':
        food_name = data.split('/')[0]  
    else:
        food_name = data.split('/')[-1]
    return food_name

In [36]:
y_train = x_train['path'].map(lambda x: split(data = x, id = 'Class'))
#y_train.head()
len(y_train)

75750

In [37]:
y_test = x_test['path'].map(lambda x: split(data = x, id = 'Class'))
len(y_test)

25250